**summarize**
- use MapReduce kind of logic

**question-answer**
- use retrieval qa chain kind of logic

**troubleshoot**
- use a retriever -> something similar to web-researcher
    - internet-search
    - internal-rag-search


In [10]:
%store -r results

In [14]:
messages = [item["@message"] for item in results]
messages = messages[::-1] # reason - list is ordered in desc based on timestamp in query
text = "\n".join(messages)
# print(text)

### summarize

In [17]:
from langchain_core.prompts import ChatPromptTemplate

summarize_prompt_template = """You are an expert in understanding logs from an application. 
Summarize the given log data in bullet points outline the main issue. Make a markdown table of error and source.
Logs: {text}"""

summarize_prompt = ChatPromptTemplate.from_template(summarize_prompt_template)

summarize_prompt.pretty_print()

================================ Human Message =================================

You are an expert in understanding logs from an application. 
Summarize the given log data in bullet points outline the main issue. Make a markdown table of error and source.
Logs: {text}


In [18]:
import boto3, json
from langchain_community.chat_models import BedrockChat

llm = BedrockChat(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    # model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    client=boto3.client("bedrock-runtime"),
    model_kwargs={"temperature": 0.0, "max_tokens":512}
)

In [19]:
summarize_chain = summarize_prompt | llm

In [21]:
response = summarize_chain.invoke({"text": text})

In [22]:
print(response.content)

Based on the provided log data, the main issue appears to be a CUDA kernel error with the message "CUDA error: invalid argument". Here's a summary of the key points:

**Outline:**

- The CUDA kernel error is reported asynchronously, so the provided stacktrace might not be accurate.
- The error occurred during the `Decode` method of the `text_generation_server.server.Server` class.
- The error occurred within the `forward` method of the `text_generation_server.models.custom_modeling.flash_rw_modeling.FlashRWTransformer` class.
- The error occurred when trying to apply the activation function (`self.act`) to the hidden states.

**Markdown Table of Error and Source:**

| Error | Source |
| --- | --- |
| RuntimeError: CUDA error: invalid argument | `text_generation_server.models.custom_modeling.flash_rw_modeling.FlashRWTransformer.forward` |

The log data suggests two potential solutions to debug the issue:

1. Compile the application with `TORCH_USE_CUDA_DSA` to enable device-side asserti

In [24]:
from IPython.display import Markdown

display(response.content)

'Based on the provided log data, the main issue appears to be a CUDA kernel error with the message "CUDA error: invalid argument". Here\'s a summary of the key points:\n\n**Outline:**\n\n- The CUDA kernel error is reported asynchronously, so the provided stacktrace might not be accurate.\n- The error occurred during the `Decode` method of the `text_generation_server.server.Server` class.\n- The error occurred within the `forward` method of the `text_generation_server.models.custom_modeling.flash_rw_modeling.FlashRWTransformer` class.\n- The error occurred when trying to apply the activation function (`self.act`) to the hidden states.\n\n**Markdown Table of Error and Source:**\n\n| Error | Source |\n| --- | --- |\n| RuntimeError: CUDA error: invalid argument | `text_generation_server.models.custom_modeling.flash_rw_modeling.FlashRWTransformer.forward` |\n\nThe log data suggests two potential solutions to debug the issue:\n\n1. Compile the application with `TORCH_USE_CUDA_DSA` to enable 

None


### question-answer

### troubleshoot